# CZ2001 Algorithms Project 1
Done By: Chua Zi Heng, Mun Kei Wuai, Pooja Srinivas Nag, Singh Jasraj, Tan Wen Xiu


Methods:
1. Brute Force
2. Two Hash Boyer Moore
3. Maximum Average Shift


In [112]:
# importing the necessary libraries 

# ! pip install fastaparser
import fastaparser
import time

import numpy as np
import fastaparser
import sys
import os 

### Load Fasta File and Sequence 

In [113]:
##################################### - INPUT FASTA DIRECTORY -  #####################################################

with open("sequence.fasta") as fasta_file:
    parser = fastaparser.Reader(fasta_file)
    for seq in parser:
        # seq is a FastaSequence object
        print("ID:", seq.id)
        print("Description:", seq.description)
        print("Sequence:", seq.sequence_as_string())
        print()
sequence = seq.sequence_as_string()

ID: NC_045512.2
Description: Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome
Sequence: ...



In [114]:
####################################### - INPUT QUERY SEQUENCE HERE - #######################################

p = "GTGGACAAATTGTCACCTGTGCAAAGGAAATT"

### Method 1: Brute Force

In [115]:
# defining the function for brute force 

def brute_force(S,p):
    
    matches = [] 
    for i in range(0,len(S)-len(p)+1):                      # iterate through each window window of length m in the text
        for j in range(0,len(p)): 
            if S[i+j] != p[j]:                              # compare the corresponding characters in the window and the pattern
                break                                       # break out of the loop if the characters do not match
        else:                
            matches.append(i)                               # executes only when the loop above does not break anywhere
    if not matches:                                         # if matches list is empty
        print("Query does not exist in sequence!")
    else:                                                   # if matches list is not empty, print the indexes
        print("Query found at index position(s):", *matches) 

#### Execution Time for Brute Force

In [116]:
start = time.time()
brute_force(sequence, p)
end = time.time()

# total time taken
print("Runtime of the Brute Force algorithm is {} s".format(round(end-start, 6)))

Query found at index position(s): 2246
Runtime of the Brute Force algorithm is 0.012029 s


### Method 2: Two-Hash Boyer-Moore

In [117]:
# define the 2 hash functions used

def Hash1(key):                                                     # DJB2 hash function using bitwise
    
    hash = 5381
    for i in range(len(key)):
        hash = ((hash << 5) + hash) + ord(key[i])
    
    return hash

def Hash2(key):                                                     # SDBMHash function using bitwise
    
    hash = 0
    for i in range(len(key)):
        hash = ord(key[i]) + (hash << 6) + (hash << 16) - hash;
    
    return hash

In [118]:
#Preprocessing and main THMB algorithm

new_ord = {"A":0, "C":1, "G":2, "T":3}

def THBM(text, pattern):
    
    m, n = len(pattern), len(text)
    alphabets = 4
    p_hash1, p_hash2 = Hash1(pattern[:m//2]), Hash2(pattern)
    result, skip = [], []

    if m>n: 
        
        return -1                                                 # if pattern longer than text

    # create skip table (Inspired by Boyer Moore Horspool)
    for _ in range(alphabets): 
        skip.append(m)                                            # if letter does not exist in pattern, store length of pattern
    for index in range(m - 1): 
        skip[new_ord.get(pattern[index])] = m-index-1             # value = length-index-1
    skip = tuple(skip)
    k = m-1                                                       # k is the last element of pattern window

    while k<n: 
        
        j = m - 1
        if text[k]==pattern[j] and text[k-j]==pattern[0]:         # Check if first and last char are the same 
            t_hash1 = Hash1(text[k-m+1:k-m+1+m//2])           
            if t_hash1 == p_hash1:                                # Calculate hash for first half of text window
                t_hash2 = Hash2(text[k-m+1:k+1])                      
                if t_hash2 == p_hash2:                            # Calculate hash for whole text window
                    result.append(k-m+1)
        k += skip[new_ord.get(text[k])]                           # Number of characters to be skipped 

    if not result:
        print("Query does not exist in sequence!")
    else:
        print("Query found at index position(s):", *result)

#### Execution time for Two-Hash Boyer-Moore

In [119]:
start = time.time()
THBM(sequence, p)
end = time.time()

# total time taken
print("Runtime of the Two Hash BM algorithm is {} s".format(round(end-start, 6)))

Query found at index position(s): 2246
Runtime of the Two Hash BM algorithm is 0.003991 s


### Method 3: Maximal Average Shift (MAS)

In [120]:
def MAS_preprocessing(query_string, length_query_string):

    # probabaility_distribution - Discrete frequency distribtuion followed by the text characters
    # scan_order - Order in which the characters in query string would compare with those in text window
    # available_pos - The set from which we can choose a position to compare at the a particular step
    # shift - Stores the shift length shift[l][s] if scan[i] = l, and the text character in this position is s
    # mas_shift - Stores the shift length shift[l][s], where l is the position that maximises this value
    # safe - shift lengths, k, such that P[scan[j]] = P[scan[j]-k], for all j<i, where is the position in the scan order we are currently trying to determine

    scan_order = []
    available_pos = list(range(1,length_query_string+1))
    shift = [{char:1 for char in probability_distribution.keys()} for _ in range(length_query_string)] 
    mas_shift = [{key:None for key in probability_distribution.keys()} for _ in range(length_query_string)] 
    safe = [1]*length_query_string 

    for i in range(1,length_query_string+1): 
        average_shift = [0]*length_query_string 
        for pos in available_pos: 
            for char in probability_distribution.keys():
                for k in range(shift[pos-1][char], length_query_string+1): 
                    if safe[k-1]==1 and (pos<k+1 or char==query_string[pos-k-1]):
                        shift[pos-1][char] = k
                        break
                average_shift[pos-1] += shift[pos-1][char]*probability_distribution[char]

        l = np.where(average_shift==np.max(average_shift))[0][0]+1 
        available_pos.remove(l)
        scan_order.append(l)
        for char in probability_distribution.keys():
            mas_shift[l-1][char] = shift[l-1][char]

        for k in range(1,l):
            if not l<k+1 and query_string[l-1]!=query_string[l-k-1]:
                safe[k-1] = 0

    return scan_order, mas_shift

def find(text_string, length_text_string, query_string, length_query_string, probability_distribution):

    scan_order, mas_shift = MAS_preprocessing(query_string, length_query_string)
    final_idx = length_text_string-length_query_string
    w = 0                               # Start position index for the window 
    output = []                         # Stores the set of indices where the query string is found

    while w <= final_idx: 

        i = 1

        # Enter this loop as long as the characters in the text and the query match
        while i<=length_query_string and text_string[w+scan_order[i-1]-1]==query_string[scan_order[i-1]-1]:
            i += 1

        # If the query string is found, append to the output, shift the window depending on the last character of the query
        if i>length_query_string:
            output.append(w) 
            w += mas_shift[scan_order[length_query_string-1]-1][text_string[w+scan_order[length_query_string-1]-1]]
        # Else just shift the window depending on the last matched character
        else:
            w += mas_shift[scan_order[i-1]-1][text_string[w+scan_order[i-1]-1]]

    if not output:
        print("No occurrence of the query string found!")
    else:
        print("Query found at index position(s):", *output)

In [121]:
probability_distribution = {"A":0.25, "T":0.25, "G":0.25, "C":0.25}
n, m = len(sequence), len(p)

#### Execution time for Maximum Average Shift

In [122]:
start = time.time()
find(sequence, n, p, m, probability_distribution)
end = time.time()

print("Runtime of the Two Hash BM algorithm is {} s".format(round(end-start, 6)))

Query found at index position(s): 2246
Runtime of the Two Hash BM algorithm is 0.011729 s
